In [1]:
import pandas as pd
import numpy as np

## Data Preparation

In [2]:
df = pd.read_csv('../data/spotify_dataset.csv', on_bad_lines='skip')

In [3]:
# Fixed Column Headers (removed quotes, whitespace, and simplified the column names)
df.columns = df.columns.str.replace("\"", "").str.strip().str.replace("name","")

In [4]:
df = df.dropna() # Removes rows with NULL values

In [5]:
# Column that combines song and artist
df['track_artist'] = df['track'] + ', ' + df['artist']

In [6]:
n_tracks = df.track_artist.nunique()
n_users = df.user_id.nunique()
n_interactions = len(df)

## Data Validation

In [7]:
# Split into training and validation datasets randomly
# Random sampling of users changes the baseline (.002...)
from sklearn.model_selection import train_test_split

df_train, df_val = train_test_split(df, test_size=0.2)

In [8]:
# Frequency of most common songs
n_rec = 10
freq = df_train.track_artist.value_counts().iloc[:n_rec].index.values

In [9]:
# Grouping Songs by User_ID
val_group = df_val.groupby('user_id').track_artist.apply(set)

In [10]:
n_val = len(val_group)
recommendations = np.repeat([freq], n_val, axis=0)

In [11]:
# Finds out how often the top 10 tracks appear in validation dataset users
def avg_prec(recommendations, val_group):
    average_precision = 0

    for i, user_id in enumerate(val_group):
        count = 0
        for track_artist in recommendations[i]:
            if track_artist in user_id:
                count = count + 1
        precision = count / n_rec
        average_precision = average_precision + precision

    average_precision = average_precision / n_val
    return average_precision

In [12]:
avg_prec(recommendations, val_group)

0.02720062410609867

From the data validation performed, we know the baseline is approximately between .026 and .028.

## Content-Based Recommendations

In [13]:
# Find out overlapping users in both datasets
# We only need to make recs for them (this number varies due to random sampling)
target_users = set(df_val.user_id) & set(df_train.user_id)
len(target_users)

15319

The number of users that we should make recs for is a bit over 15000, as they exist in both the traning and validation datasets.

In [14]:
df_train_subset = df_train[df_train.user_id.isin(target_users)]

Now we create vectorizors for turning tracks and users into vectors

In [15]:
# Sklearn Imports
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

from sklearn.base import TransformerMixin, BaseEstimator

from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.preprocessing import normalize

In [16]:
class ToDictTranformer(TransformerMixin, BaseEstimator):
    def __init__(self):
        self.columns = None
    
    def fit(self, df, y=None):
        self.columns = df.columns
        return self

    def transform(self, df):
        return df.to_dict(orient='records')

    def get_feature_names_out(self, *args, **kwargs):
        return self.columns

In [17]:
ohe_columns = ['track_artist']

categorical_pipeline = make_pipeline(ToDictTranformer(), DictVectorizer())

vectorizer = ColumnTransformer([
    ('categorical', categorical_pipeline, ohe_columns)
])

In [18]:
vectorizer.fit(df_train)

ColumnTransformer(transformers=[('categorical',
                                 Pipeline(steps=[('todicttranformer',
                                                  ToDictTranformer()),
                                                 ('dictvectorizer',
                                                  DictVectorizer())]),
                                 ['track_artist'])])

In [19]:
# Representing each track_artist as a unit vector
x_ta = vectorizer.transform(df)
x_ta = normalize(x_ta)

For each user we will:
- Represent each user and their average vector of track+artist(song) they have in their playlists
- Compute the similarity between the user vector and each song vector
- Return top 10 similar songs (This will be the recommendation for each user)

In [20]:
user_recommendations = {}

for user_id, df in df_train_subset.groupby('user_id'):
    # represent each user as an average of all the songs added in their playlists
    x_user = vectorizer.transform(df)
    
    x_user = x_user.sum(axis=0)
    x_user = np.asarray(x_user)
    x_user = normalize(x_user).reshape(-1)
    
    # compute the similarity between the user and songs
    scores = x_ta.dot(x_user)

    argidx = scores.argpartition(np.arange(10))[:10]

    user_recommendations[user_id] = argidx
    print(user_recommendations)

{'00055176fea33f6e027cd3302289378b': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64)}
{'00055176fea33f6e027cd3302289378b': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '0007f3dd09c91198371454c608d47f22': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64)}
{'00055176fea33f6e027cd3302289378b': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '0007f3dd09c91198371454c608d47f22': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '000b0f32b5739f052b9d40fcc5c41079': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64)}
{'00055176fea33f6e027cd3302289378b': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '0007f3dd09c91198371454c608d47f22': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '000b0f32b5739f052b9d40fcc5c41079': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '000c11a16c89aa4b14b328080f5954ee': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64)}
{'00055176fea33f6e027cd3302289378b': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '0007f3dd09c91198371454c608

{'00055176fea33f6e027cd3302289378b': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '0007f3dd09c91198371454c608d47f22': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '000b0f32b5739f052b9d40fcc5c41079': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '000c11a16c89aa4b14b328080f5954ee': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '00123e0f544dee3ab006aa7f1e5725a7': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '00139e9cb50fb309549e1561b476226d': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '00154ec9dd1acd4ebfb521629dcb3948': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '001599a07cb8ef5f114a9fcf4e0e2757': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '0019363a0d57e94d39988c31eeb8d015': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '001c4e5b73eca68ee9756bb0c7d2f855': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '0025022960e5f0d7d01af5d840014594': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '002544ea04896444d79d3e4a7e0734

{'00055176fea33f6e027cd3302289378b': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '0007f3dd09c91198371454c608d47f22': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '000b0f32b5739f052b9d40fcc5c41079': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '000c11a16c89aa4b14b328080f5954ee': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '00123e0f544dee3ab006aa7f1e5725a7': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '00139e9cb50fb309549e1561b476226d': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '00154ec9dd1acd4ebfb521629dcb3948': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '001599a07cb8ef5f114a9fcf4e0e2757': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '0019363a0d57e94d39988c31eeb8d015': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '001c4e5b73eca68ee9756bb0c7d2f855': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '0025022960e5f0d7d01af5d840014594': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '002544ea04896444d79d3e4a7e0734

{'00055176fea33f6e027cd3302289378b': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '0007f3dd09c91198371454c608d47f22': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '000b0f32b5739f052b9d40fcc5c41079': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '000c11a16c89aa4b14b328080f5954ee': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '00123e0f544dee3ab006aa7f1e5725a7': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '00139e9cb50fb309549e1561b476226d': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '00154ec9dd1acd4ebfb521629dcb3948': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '001599a07cb8ef5f114a9fcf4e0e2757': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '0019363a0d57e94d39988c31eeb8d015': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '001c4e5b73eca68ee9756bb0c7d2f855': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '0025022960e5f0d7d01af5d840014594': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '002544ea04896444d79d3e4a7e0734

{'00055176fea33f6e027cd3302289378b': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '0007f3dd09c91198371454c608d47f22': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '000b0f32b5739f052b9d40fcc5c41079': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '000c11a16c89aa4b14b328080f5954ee': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '00123e0f544dee3ab006aa7f1e5725a7': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '00139e9cb50fb309549e1561b476226d': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '00154ec9dd1acd4ebfb521629dcb3948': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '001599a07cb8ef5f114a9fcf4e0e2757': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '0019363a0d57e94d39988c31eeb8d015': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '001c4e5b73eca68ee9756bb0c7d2f855': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '0025022960e5f0d7d01af5d840014594': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '002544ea04896444d79d3e4a7e0734

...

{'00055176fea33f6e027cd3302289378b': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '0007f3dd09c91198371454c608d47f22': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '000b0f32b5739f052b9d40fcc5c41079': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '000c11a16c89aa4b14b328080f5954ee': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '00123e0f544dee3ab006aa7f1e5725a7': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '00139e9cb50fb309549e1561b476226d': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '00154ec9dd1acd4ebfb521629dcb3948': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '001599a07cb8ef5f114a9fcf4e0e2757': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '0019363a0d57e94d39988c31eeb8d015': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '001c4e5b73eca68ee9756bb0c7d2f855': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '0025022960e5f0d7d01af5d840014594': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '002544ea04896444d79d3e4a7e0734

{'00055176fea33f6e027cd3302289378b': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '0007f3dd09c91198371454c608d47f22': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '000b0f32b5739f052b9d40fcc5c41079': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '000c11a16c89aa4b14b328080f5954ee': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '00123e0f544dee3ab006aa7f1e5725a7': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '00139e9cb50fb309549e1561b476226d': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '00154ec9dd1acd4ebfb521629dcb3948': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '001599a07cb8ef5f114a9fcf4e0e2757': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '0019363a0d57e94d39988c31eeb8d015': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '001c4e5b73eca68ee9756bb0c7d2f855': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '0025022960e5f0d7d01af5d840014594': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '002544ea04896444d79d3e4a7e0734

{'00055176fea33f6e027cd3302289378b': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '0007f3dd09c91198371454c608d47f22': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '000b0f32b5739f052b9d40fcc5c41079': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '000c11a16c89aa4b14b328080f5954ee': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '00123e0f544dee3ab006aa7f1e5725a7': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '00139e9cb50fb309549e1561b476226d': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '00154ec9dd1acd4ebfb521629dcb3948': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '001599a07cb8ef5f114a9fcf4e0e2757': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '0019363a0d57e94d39988c31eeb8d015': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '001c4e5b73eca68ee9756bb0c7d2f855': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '0025022960e5f0d7d01af5d840014594': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '002544ea04896444d79d3e4a7e0734

{'00055176fea33f6e027cd3302289378b': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '0007f3dd09c91198371454c608d47f22': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '000b0f32b5739f052b9d40fcc5c41079': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '000c11a16c89aa4b14b328080f5954ee': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '00123e0f544dee3ab006aa7f1e5725a7': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '00139e9cb50fb309549e1561b476226d': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '00154ec9dd1acd4ebfb521629dcb3948': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '001599a07cb8ef5f114a9fcf4e0e2757': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '0019363a0d57e94d39988c31eeb8d015': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '001c4e5b73eca68ee9756bb0c7d2f855': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '0025022960e5f0d7d01af5d840014594': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), '002544ea04896444d79d3e4a7e0734

KeyboardInterrupt: 

Now we create a matrix with predictions so that we can see if the average precision is better than our baseline.

In [24]:
n_val = len(val_group)
cb_recs = np.repeat([freq], n_val, axis=0)

In [25]:
val_group.index

Index(['00055176fea33f6e027cd3302289378b', '0007f3dd09c91198371454c608d47f22',
       '000b0f32b5739f052b9d40fcc5c41079', '000c11a16c89aa4b14b328080f5954ee',
       '00123e0f544dee3ab006aa7f1e5725a7', '00139e9cb50fb309549e1561b476226d',
       '00154ec9dd1acd4ebfb521629dcb3948', '001599a07cb8ef5f114a9fcf4e0e2757',
       '0019363a0d57e94d39988c31eeb8d015', '001c4e5b73eca68ee9756bb0c7d2f855',
       ...
       'ffc74c046f6f0a845db4188d9a445fb2', 'ffc9e5f9869491983a67f254ab6e61a6',
       'ffd0d3573b8e8c5da39f743828ec4840', 'ffd6a7efcb3afe1452c4088ee2d3ede0',
       'ffe11226cdea81a2db9262c0ec7f5d71', 'ffe32d5412269f3041c58cbf0dde3306',
       'ffec270eae226caa14ddaef291d73fff', 'fff60baf392613ed33f745b89a9b38f7',
       'fff616055993498d6127f3f467cf9f2b', 'fff77dadf8528083c920b9c018847e8b'],
      dtype='object', name='user_id', length=15382)

In [26]:
for i, user_id in enumerate(val_group.index):
    if user_id in user_recommendations:
        cb_recs[i] = user_recommendations[user_id]

In [28]:
avg_prec(cb_recs, val_group)

0.026921076583019725